In [4]:
import sqlite3

# Path to your SQLite database
db_path = '/home/jovyan/ECON481/auctions.db'

# Connect to the SQLite database
conn = sqlite3.connect(db_path)

# Create a cursor object to interact with the database
cursor = conn.cursor()

# Example query: List all tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Print out the tables
for table in tables:
    print(table[0])

# Remember to close the connection when done
conn.close()


bids
items


In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

path = 'auctions.db'
path2 = 'auctions2.db'
path3 = 'auctions3.db'

class DataBase:
    def __init__(self, loc: str, db_type: str = "sqlite") -> None:
        """Initialize the class and connect to the database"""
        self.loc = loc
        self.db_type = db_type
        self.engine = create_engine(f'{self.db_type}:///{self.loc}')
    def query(self, q: str) -> pd.DataFrame:
        """Run a query against the database and return a DataFrame"""
        with Session(self.engine) as session:
            df = pd.read_sql(q, session.bind)
        return(df)

auctions = DataBase(path)
auctions2 = DataBase(path2)
auctions3 = DataBase(path3)

In [179]:
query = "SELECT * FROM bids;"
pd.set_option('display.max_rows', 10)
print(auctions.query(query))

     index  bidLogId     itemId  itemPrice  bidAmount  \
0       50         0  178348858       9.99       20.0   
1       51         0  178348858      13.00       12.0   
2       52         0  178348858      21.00       23.0   
3       53         0  178348858      24.00       35.0   
4       54         0  178348858      36.00       48.0   
..     ...       ...        ...        ...        ...   
546  29510         0  177920252      17.00       47.0   
547  29511         0  177920252      24.50       22.5   
548  29512         0  177920252      32.00       30.0   
549  29513         0  177920252      41.00       39.0   
550  29514         0  177920252      47.00       45.0   

                        bidTime  quantity bidIPAddress adCode serverIP  \
0    2023-09-18 16:11:04.587000         1         None   None     None   
1    2023-09-22 14:22:06.700000         1         None   None     None   
2    2023-09-23 12:35:18.157000         1         None   None     None   
3    2023-09-23 18:

In [233]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
def std() -> str:
    q = """
        select itemid, 
        sqrt(sum((bidAmount - avg_amount) * (bidAmount - avg_amount)) / (count - 1)) as std
        from (
            select itemid, bidAmount,
                avg(bidAmount) over (partition by itemid) as avg_amount,
                count(bidAmount) over (partition by itemid) as count
            from bids
            where bidAmount is not null
        )
        where count > 1
        group by itemid
        """
    return q

print(auctions.query(std()))

       itemid       std
0   172998011  0.000000
1   174445924  0.577350
2   174551689  1.830034
3   174767945  8.607399
4   174871788  3.109126
..        ...       ...
82  182571641  7.767700
83  182660924  4.364634
84  182745253  2.884098
85  182883873  0.707107
86  182925212  1.154701

[87 rows x 2 columns]


In [190]:
def p(itemid: int) -> str:
    q = f"""
    SELECT itemid, AVG(bidAmount) as avg_amount,
        COUNT(bidAmount) as count,
        GROUP_CONCAT(bidAmount) as all_bid_amounts,
        sum(bidAmount) as sum
    FROM bids
    WHERE bidAmount IS NOT NULL AND itemid = {itemid};
    """
    return q
print(auctions.query(p(174445924)))


      itemId  avg_amount  count all_bid_amounts   sum
0  174445924   15.333333      3  15.0,15.0,16.0  46.0


In [234]:
def bidder_spend_frac() -> str:
    q = """
    select 
        bidderName,
        sum(case when highBidderName = bidderName then bidAmount else 0 end) as total_spend,
        sum(max) as total_bids,
        sum(case when highBidderName = bidderName then bidAmount else 0 end) / sum(max) as spend_frac
    from (
        select bidderName, itemid, highBidderName, bidAmount, max(bidAmount) as max
        from bids
        group by bidderName, itemid
    )
    group by bidderName;
    """
    return q

print(auctions.query(bidder_spend_frac()))

    bidderName  total_spend  total_bids  spend_frac
0       1****3        28.00       28.00    1.000000
1       2****h         0.00       35.00    0.000000
2       4****S         0.00       23.99    0.000000
3       7****1        23.99       23.99    1.000000
4       A****3        15.00      383.00    0.039164
..         ...          ...         ...         ...
279     v****p         9.99        9.99    1.000000
280     w****5        46.99       46.99    1.000000
281     w****7        19.99       19.99    1.000000
282     y****2        13.01       13.01    1.000000
283     y****a        34.99       34.99    1.000000

[284 rows x 4 columns]


In [160]:
def get_bid_amounts(itemid: int) -> str:
    q = f"""
    SELECT bidAmount
    FROM bids
    WHERE itemid = {itemid};
    """
    return q

itemid = 174445924          # Replace 123 with the specific itemid you're interested in
print(auctions.query(get_bid_amounts(itemid)))

   bidAmount
0       15.0
1       15.0
2       16.0


In [4]:
def min_increment_freq() -> str:
    q = """
    select cast(sum(case when b.bidAmount - p.prevbid = i.bidIncrement then 1 else 0 end) as float) / nullif(count(*),0) as freq
    from bids as b
    inner join items as i on b.itemId = i.itemId
    inner join (
        select itemId, bidTime, lag(bidAmount) over (partition by itemId order by bidTime) as prevbid
        from bids
    ) as p on b.itemId = p.itemId and b.bidTime = p.bidTime
    where i.isBuyNowUsed != 1;
    """
    return q

print(auctions.query(min_increment_freq()))

       freq
0  0.108856


In [235]:
from sqlalchemy import func, distinct
def win_perc_by_timestamp() -> str:
    q = """
    /* stores auction info */
    with a as (
        select itemid, starttime, endtime
        from items
    ), 
    /* figures out the timestamp_bin and which bin it belongs in */
    time as (
        select b.itemid, 
            cast(((julianday(a.endtime) - julianday(b.bidtime)) / 
                (julianday(a.endtime) - julianday(a.starttime)) * 10) + 1 as int) as timestamp_bin,
            max(b.bidAmount) over (partition by b.itemid) as max
        from bids as b
        inner join items as a on b.itemid = a.itemid
    ),
    /* counts the number of items are within each bin */
    win as (
        select time.timestamp_bin, 
        count(distinct b.itemid) as win_count
        from time
        inner join bids as b on time.itemid = b.itemid
                            and time.max = b.bidAmount
        inner join a on b.itemid = a.itemid
        where cast(((julianday(a.endtime) - julianday(b.bidtime)) / 
                   (julianday(a.endtime) - julianday(a.starttime)) * 10) + 1 as int) = time.timestamp_bin
        group by time.timestamp_bin
    )
    /* outputs the bins and the winning percentage from within each bin*/
    select timestamp_bin,
        win_count * 1.0 / (select sum(win_count) from win) as win_perc
    from win
    group by timestamp_bin
    order by timestamp_bin;
    """
    return q

print(auctions.query(win_perc_by_timestamp()))

   timestamp_bin  win_perc
0              1  0.744318
1              2  0.102273
2              3  0.028409
3              4  0.034091
4              5  0.034091
5              6  0.005682
6              8  0.011364
7              9  0.022727
8             10  0.017045


In [207]:
q = """
with a as (
    select itemid, starttime, endtime 
    from items
)
select b.itemid,
    cast(((julianday(a.endtime) - julianday(b.bidtime)) / (julianday(a.endtime) - julianday(a.starttime)) * 10) + 1 as int) as timestamp_bin, max(b.bidAmount) as max
from bids as b
inner join items as a on b.itemid = a.itemid
where (b.itemId, b.bidAmount) in (
    select itemId, max(bidAmount)
    from bids
    group by itemId
)
group by b.itemid, timestamp_bin
"""
df = auctions.query(q)
print(df)

        itemId  timestamp_bin    max
0    165561698              1   9.91
1    170983900              1   9.91
2    172998011              1   9.91
3    172998011              2   9.91
4    173907435              1  14.99
..         ...            ...    ...
171  182760698              1  16.99
172  182777527              1  15.99
173  182883873              1  11.99
174  182894197              1   8.99
175  182925212              1  10.01

[176 rows x 3 columns]


In [232]:
def github() -> str:
    return "https://github.com/kalvinlee/ECON481/blob/main/problemset6.py"

github()

'https://github.com/kalvinlee/ECON481/blob/main/problemset6.py'